In [1]:
#@title MIT License
#
# Copyright (c) 2020 Balázs Pintér
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

In [2]:
import numpy as np
import scipy
import scipy.sparse as sps
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import tensorflow as tf
from nltk.corpus import movie_reviews

2024-11-19 14:25:42.552615: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-19 14:25:42.553431: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-19 14:25:42.555812: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-19 14:25:42.562584: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732026342.574394   38332 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732026342.57

In [3]:
num_neurons = 20
num_features = 5000

In [4]:
import nltk
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [5]:
# getting the data, like last time

corpus, targets = zip(*[(movie_reviews.raw(fileid), category)
                         for category in movie_reviews.categories() for fileid in movie_reviews.fileids(category)])

count_vectorizer = CountVectorizer(stop_words='english', max_df=0.95, min_df=2, max_features=num_features)
bows = count_vectorizer.fit_transform(corpus)

# convert targets to numbers
targets = np.array([0 if target == 'neg' else 1 for target in targets])

bows = bows.astype(np.float32)
targets = targets.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(bows, targets, test_size=0.1, shuffle=True)

# the problem: we have sparse arrays, but neural network need dense arrays!
# the solution will be word embeddings, here we just convert to dense arrays
X_train = X_train.toarray()
X_test = X_test.toarray()

In [6]:
# in TensorFlow 2.0, we can see the tensor instantly (same as eager mode in TensorFlow 1.0)
tf.ones((4, 4))

2024-11-19 14:25:45.334280: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)>

In [7]:
# we also see the results of an operation instantly
tf.add(tf.constant([1, 2, 3]), tf.constant([4, 5, 6]))

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([5, 7, 9], dtype=int32)>

In [8]:
tf.zeros((4, 4)) + 4

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[4., 4., 4., 4.],
       [4., 4., 4., 4.],
       [4., 4., 4., 4.],
       [4., 4., 4., 4.]], dtype=float32)>

In [22]:
# variables are for the parameters (weights) for the NN
W1 = tf.Variable(tf.random.normal((num_neurons, num_features), stddev=0.01), name='W1')
b1 = tf.Variable(tf.random.normal((num_neurons, 1), stddev=0.01), name='b1')
W15 = tf.Variable(tf.random.normal((num_neurons, num_neurons), stddev=0.01), name='W15')
b15 = tf.Variable(tf.random.normal((num_neurons, 1), stddev=0.01), name='b15')
W2 = tf.Variable(tf.random.normal((1, num_neurons), stddev=0.01), name='W2')
b2 = tf.Variable(tf.random.normal((1, 1), stddev=0.01), name='b2')
W1, b1, W2, b2

(<tf.Variable 'W1:0' shape=(20, 5000) dtype=float32, numpy=
 array([[ 0.00055755,  0.01734037, -0.01524723, ...,  0.00191359,
         -0.00475368, -0.00106215],
        [-0.00899945, -0.01513705,  0.00947089, ..., -0.00455601,
          0.00875724,  0.00752879],
        [ 0.00485277,  0.01038518, -0.00593169, ...,  0.01099314,
          0.00319859, -0.01897568],
        ...,
        [-0.00926714,  0.00126028, -0.01369052, ...,  0.0137404 ,
         -0.00135495, -0.00367781],
        [-0.00779927,  0.00469138,  0.01839181, ..., -0.00223047,
         -0.00155172,  0.01317024],
        [ 0.00353124, -0.0007916 , -0.01210199, ...,  0.00124528,
          0.0017817 ,  0.0059835 ]], dtype=float32)>,
 <tf.Variable 'b1:0' shape=(20, 1) dtype=float32, numpy=
 array([[-0.01236881],
        [ 0.01129107],
        [-0.00497256],
        [-0.01576513],
        [ 0.00188694],
        [-0.01615887],
        [ 0.00146614],
        [-0.01671192],
        [-0.01026485],
        [ 0.00751555],
        [-

In [23]:
# the network itself
def forward_pass(X):
    z = tf.add(tf.matmul(W1, X), b1)
    a = tf.nn.relu(z)
    z = tf.add(tf.matmul(W15, a), b15)
    a = tf.nn.relu(z)
    z = tf.add(tf.matmul(W2, a), b2)
    #a = tf.nn.sigmoid(z) # already in loss function
    return z

In [24]:
# the loss function
def loss(a, y):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a, labels=y))

In [25]:
def minibatches(X, y, minibatch_size):

# shuffling
    perm = np.random.permutation(X.shape[0])
    X_shuffled = X[perm]
    y_shuffled = y[perm]

    mini_batches = [(X_shuffled[i*minibatch_size:(i+1)*minibatch_size], y_shuffled[i*minibatch_size:(i+1)*minibatch_size])
                   for i in range(X_shuffled.shape[0] // minibatch_size)]
# we lost some examples at the end, doesn't really matter for this example
    return mini_batches

In [26]:
num_epochs = 10

$w_{11} := w_{11} - lr*\frac{loss}{\delta w_{11}}$

In [27]:
learning_rate = 0.3
minibatch_size = 32
for epoch in range(num_epochs):
# minibatch training
    epoch_loss = 0
    num_minibatches = X_train.shape[0] // minibatch_size
    for X_mini, y_mini in minibatches(X_train, y_train, minibatch_size):
# we use a GradientTape to record the gradient for each minibatch
        with tf.GradientTape() as t:
            # important: we have to transpose here! each example is in a column
            mini_loss = loss(forward_pass(X_mini.T), y_mini[None, :])
        # update the weights
        dW1, db1, dW15, db15, dW2, db2 = t.gradient(mini_loss, [W1, b1, W15, b15, W2, b2])
        W1.assign_sub(learning_rate * dW1)
        b1.assign_sub(learning_rate * db1)
        W15.assign_sub(learning_rate * dW15)
        b15.assign_sub(learning_rate * db15)
        W2.assign_sub(learning_rate * dW2)
        b2.assign_sub(learning_rate * db2)
        epoch_loss += mini_loss
    epoch_loss /= num_minibatches
    predictions = forward_pass(X_train.T)
    predictions = np.array([0 if pred < 0 else 1 for pred in tf.squeeze(predictions)])
    accuracy = (y_train == predictions).sum() / len(y_train)
    print("Loss in epoch {}: {}, accuracy: {}".format(epoch, epoch_loss, accuracy))
# accuracy on test set
predictions = forward_pass(X_test.T)
predictions = np.array([0 if pred < 0 else 1 for pred in tf.squeeze(predictions)])
accuracy = (y_test == predictions).sum() / len(y_test)
print("Accuracy on test set: {}".format(accuracy))

Loss in epoch 0: 0.6937556266784668, accuracy: 0.5016666666666667
Loss in epoch 1: 0.6942073702812195, accuracy: 0.5016666666666667
Loss in epoch 2: 0.6941079497337341, accuracy: 0.5016666666666667
Loss in epoch 3: 0.693834125995636, accuracy: 0.49833333333333335
Loss in epoch 4: 0.6931800842285156, accuracy: 0.49833333333333335
Loss in epoch 5: 0.6551679372787476, accuracy: 0.8144444444444444
Loss in epoch 6: 0.490001916885376, accuracy: 0.8027777777777778
Loss in epoch 7: 0.3423937261104584, accuracy: 0.9538888888888889
Loss in epoch 8: 0.38477450609207153, accuracy: 0.9527777777777777
Loss in epoch 9: 0.3311988413333893, accuracy: 0.9422222222222222
Accuracy on test set: 0.8
